# Feature Selection & Scaling

@roman

11 Mar, 2024

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.linear_model import LassoCV, lasso_path
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import shap

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Settings
# show 100 columns in pandas
pd.set_option('display.max_columns', 500)
TODAY = pd.to_datetime('today')

---
# Lasso

## Read

In [3]:
# important columns
cols_to_stay = [
    'observation_id',
    'price',
    'half_bathrooms',
    'full_bathrooms',
    'age_in_months',
    'parking_lots',
    'valuation_date',
    'sidewalk_id',
    'conservation_status_id',
    'elevator_service_id',
    'road_materials_id',
    'property_type_id',
    'level',
    'total_levels',
    'bedrooms',
    'built_area',
    'saleable_area',
    'land_area',
    'remaining_useful_life',
    # 'ppsm_comps',
    # 'ppsm_lower_comps',
    # 'ppsm_upper_comps',
    'count_supermarkets_at_1km',
    'count_hospitals_at_5km',
    'count_metro_at_1km',
    'count_schools_at_1km',
    'count_restaurants_at_1km',
    'tam_loc',
    'est_socio',
    'clase_hog',
    'sexo_jefe',
    'edad_jefe',
    'educa_jefe',
    'tot_integ',
    'mayores',
    'menores',
    'p12_64',
    'p65mas',
    'percep_ing',
    'ing_cor',
    'estim_alqu',
    'otros_ing',
    'gasto_mon',
    'vivienda',
    'alquiler',
    'pred_cons',
    'cuidados',
    'tipo_viv',
    'antiguedad',
    'cuart_dorm',
    'tenencia',
    'renta',
    'estim_pago',
    'pago_viv',
    'pago_mesp',
    'tipo_adqui',
    'viv_usada',
    'tipo_finan',
    'escrituras',
    'tot_resid',
    'tot_hom',
    'tot_muj',
    'tot_hog',
    'total_viviendas',
    'ppsm_terrain',
    # 'ppsm_terrain_lower',
    # 'ppsm_terrain_upper'
]

# read database
df_properties = pd.read_parquet("../../data/clean/properties_shif.parquet").loc[:, cols_to_stay]

# set observation_id as index
df_properties = df_properties.set_index('observation_id')

# see
print(df_properties.shape)
df_properties.head(2)

(1908483, 61)


,price,half_bathrooms,full_bathrooms,age_in_months,parking_lots,valuation_date,sidewalk_id,conservation_status_id,elevator_service_id,road_materials_id,property_type_id,level,total_levels,bedrooms,built_area,saleable_area,land_area,remaining_useful_life,count_supermarkets_at_1km,count_hospitals_at_5km,count_metro_at_1km,count_schools_at_1km,count_restaurants_at_1km,tam_loc,est_socio,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,mayores,menores,p12_64,p65mas,percep_ing,ing_cor,estim_alqu,otros_ing,gasto_mon,vivienda,alquiler,pred_cons,cuidados,tipo_viv,antiguedad,cuart_dorm,tenencia,renta,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,escrituras,tot_resid,tot_hom,tot_muj,tot_hog,total_viviendas,ppsm_terrain
observation_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43c375b2-75fd-4fcc-b311-55b396e7cbbf,2349000,1,2,204,1,2022-11-15,0,4,0,2,2,3,2,3,141,154,198,53,2,3,0,0,19,1.210938,2.724609,2,1,52.34375,7.06250,3.197266,2.740234,0.456055,2.367188,0.373291,2.210938,81674.453125,10253.206055,31.565142,47140.269531,3390.347168,1129.408691,295.455017,2439.210693,1.0,24.31250,2.015625,4,389.655090,3491.472412,512.813477,0.279052,1.0,0.174438,1,1,3.255859,1.561523,1.694336,1.019531,294870,3740
f95c14fe-9acc-4b5c-872b-bd15d412a65d,424000,0,1,192,1,2022-11-01,0,4,2,3,2,3,1,2,52,54,133,44,0,0,0,0,0,2.117188,1.929688,2,1,46.31250,5.65625,4.042969,3.228516,0.815430,2.964844,0.262451,2.564453,54447.476562,4833.395996,3.735561,37908.296875,1927.791382,391.177155,157.040634,1460.746460,1.0,17.65625,1.934570,4,133.639404,1646.661743,567.759888,0.311107,1.0,0.101379,1,1,4.058594,2.070312,1.990234,1.002930,34959,886


## Wrangling

### NaNs

In [4]:
# nan values
df_properties.isna().mean()[df_properties.isna().mean().gt(0)]

Series([], dtype: float64)

### Casting

In [5]:
# cast to string
cols_categorical = list(set(
    df_properties.select_dtypes(include='string').columns.tolist()
    + df_properties.select_dtypes(include='category').columns.tolist() 
    + df_properties.select_dtypes(include='object').columns.tolist()
    + df_properties.filter(like='_id').columns.tolist()
    ))

# to categorical
df_properties[cols_categorical] = df_properties[cols_categorical].astype('category')

## Feature Engineering

In [6]:
# params
first_date_obs = df_properties['valuation_date'].min()
last_date_obs = df_properties['valuation_date'].max()

# create columns
df_properties = (
    df_properties
    .assign(
        # objective variable
        ppsm=lambda x: x['price'] / x['saleable_area'],
        # quarters since appraisal
        quarters_since_first_appraisal=lambda x: (x['valuation_date'] - first_date_obs).dt.days / (30.4 * 3),
        # recategorizations
        has_elevator=lambda x: np.where(x['elevator_service_id'].eq(1), 1, 0),
        is_new=lambda x: x['age_in_months'].le(1).astype(int),
        conservacion_recat=lambda x: x['conservation_status_id'].replace({7: 3.5}),
    )
)

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_90355/4052324497.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  conservacion_recat=lambda x: x['conservation_status_id'].replace({7: 3.5}),


In [7]:
# count of unique values 
def count_unique_values(df, col, max_values=10):
    """
    Count the number of unique values in a column and return the result as a DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        col (str): The column to analyze.
        max_values (int): The maximum number of unique values to display. If there are more unique values, they will be grouped in an 'Others' category.
    
    Returns:
        pd.DataFrame: A DataFrame with the count of unique values.
    """
    # counts
    table_counts = (
        df[col]
        .value_counts(dropna=False)
        .reset_index()
        .assign(
            perc=lambda x: x['count'] / x['count'].sum()
        )
        .sort_values(by='count', ascending=False)
        .head(max_values)
    )
    
    num_others = df.shape[0] - table_counts['count'].sum()

    # append others if necessary
    if num_others > 0:
        table_counts = pd.concat([
            table_counts,
            pd.DataFrame({
                col: 'others',
                'count': [num_others],
                'perc': [num_others / df.shape[0]]
            })
        ],
        axis=0,
        ignore_index=True
    )

    return table_counts

# counts of categorical columns
cols_to_count = df_properties.select_dtypes(include='category').columns.tolist()
for col in cols_to_count:
    print(f"\n{col} {'=' * 50}")
    print(count_unique_values(df_properties, col, 10))


sidewalk_id ==================================================
  sidewalk_id    count      perc
0           1  1228728  0.643824
1           0   634363  0.332391
2           5    24912  0.013053
3           4    11234  0.005886
4           3     8081  0.004234
5           2     1165  0.000610

conservation_status_id ==================================================
  conservation_status_id   count      perc
0                      6  910747  0.477210
1                      4  870514  0.456129
2                      5   87763  0.045986
3                      7   35335  0.018515
4                      3    4124  0.002161

elevator_service_id ==================================================
  elevator_service_id    count      perc
0                   2  1407355  0.737421
1                   0   460853  0.241476
2                   1    40275  0.021103

road_materials_id ==================================================
  road_materials_id    count      perc
0                 2  109297

In [8]:
# see new var
df_properties.describe()

/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:731: RuntimeWarning: invalid value encountered in scalar divide
  the_mean = the_sum / count if count > 0 else np.nan
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/ravj/opt/anaconda3/envs/mds-research-

,price,half_bathrooms,full_bathrooms,age_in_months,parking_lots,valuation_date,level,total_levels,bedrooms,built_area,saleable_area,land_area,remaining_useful_life,count_supermarkets_at_1km,count_hospitals_at_5km,count_metro_at_1km,count_schools_at_1km,count_restaurants_at_1km,tam_loc,est_socio,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,mayores,menores,p12_64,p65mas,percep_ing,ing_cor,estim_alqu,otros_ing,gasto_mon,vivienda,alquiler,pred_cons,cuidados,tipo_viv,antiguedad,cuart_dorm,tenencia,renta,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,escrituras,tot_resid,tot_hom,tot_muj,tot_hog,total_viviendas,ppsm_terrain,ppsm,quarters_since_first_appraisal,has_elevator,is_new
count,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1908483,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1908483.0,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06,1.908483e+06
mean,1.483767e+06,3.491700e-01,1.488231e+00,5.291538e+01,1.135525e+00,2021-06-26 07:39:58.221204736,3.443452e+00,1.592710e+00,2.356046e+00,8.559633e+01,9.101616e+01,1.116373e+02,5.870005e+01,7.675478e-01,2.313320e+00,3.605010e-02,6.813841e-02,1.583942e+00,NaN,NaN,2.001783e+00,1.007037e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.397008e+04,7.525788e+03,5.501279e+01,3.862198e+04,4.140751e+03,1.615227e+03,2.820286e+02,1.754710e+03,NaN,NaN,NaN,3.855138e+00,5.531334e+02,2.563890e+03,5.042402e+02,3.304807e-01,1.311779e+00,NaN,3.439830e+00,1.060667e+00,NaN,NaN,NaN,NaN,1.745507e+05,4.893699e+03,1.479020e+04,9.948678e+00,2.110315e-02,4.691349e-01
min,2.000000e+05,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2019-01-01 00:00:00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+01,3.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.734375e+01,2.949219e+00,2.000000e+00,1.666992e+00,0.000000e+00,1.333008e+00,0.000000e+00,1.307617e+00,1.846488e+04,2.322577e+03,0.000000e+00,1.298909e+04,5.410000e+02,0.000000e+00,0.000000e+00,3.696533e+02,1.0,3.800781e+00,1.200195e+00,1.000000e+00,0.000000e+00,8.000000e+02,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,8.334961e-01,5.000000e-01,1.000000e+00,8.800000e+02,1.590000e+02,4.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.450000e+05,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2020-03-17 00:00:00,3.000000e+00,1.000000e+00,2.000000e+00,4.900000e+01,5.000000e+01,6.400000e+01,5.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.042969e+00,2.318359e+00,2.000000e+00,1.000000e+00,4.787500e+01,6.066406e+00,3.257812e+00,2.695312e+00,5.200195e-01,2.398438e+00,2.082520e-01,2.101562e+00,5.190688e+04,5.304737e+03,4.028208e+00,3.170567e+04,2.947570e+03,7.516017e+02,1.184212e+02,1.403711e+03,1.0,1.554688e+01,1.970703e+00,4.000000e+00,2.522204e+02,1.809421e+03,2.551812e+02,2.445493e-01,1.000000e+00,8.123779e-02,1.000000e+00,1.000000e+00,3.289062e+00,1.574219e+00,1.684570e+00,1.001953e+00,6.246800e+04,1.973000e+03,9.883721e+03,4.835526e+00,0.000000e+00,0.000000e+00
50%,8.250000e+05,0.000000e+00,1.000000e+00,8.000000e+00,1.000000e+00,2021-05-27 00:00:00,3.000000e+00,1.000000e+00,2.000000e+00,6.500000e+01,6.600000e+01,9.400000e+01,6.000000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.325195e+00,2.595703e+00,2.000000e+00,1.000000e+00,5.009375e+01,6.4804

## Scaling

In [24]:
# scaling
df_properties = (
    df_properties
    .assign(
        # surfaces
        saleable_area=lambda x: np.log(x['saleable_area']),
        land_area=lambda x: np.log1p(x['land_area']),
        # counts
        count_supermarkets_at_1km=lambda x: np.sqrt(x['count_supermarkets_at_1km']),
        count_hospitals_at_5km=lambda x: np.sqrt(x['count_hospitals_at_5km']),
        count_metro_at_1km=lambda x: np.sqrt(x['count_metro_at_1km']),
        count_schools_at_1km=lambda x: np.sqrt(x['count_schools_at_1km']),
        count_restaurants_at_1km=lambda x: np.sqrt(x['count_restaurants_at_1km']),
        # ing
        ing_cor=lambda x: np.log(x['ing_cor']),
        # terrain
        ppsm_terrain=lambda x: np.log(x['ppsm_terrain']),
        # objective variable
        log_price_per_sqm=lambda x: np.log(x['ppsm']),
    )
)

/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:731: RuntimeWarning: invalid value encountered in scalar divide
  the_mean = the_sum / count if count > 0 else np.nan
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/ravj/opt/anaconda3/envs/mds-research-

,mean,min,max
price,1483767.13341,200000.0,89589000.0
half_bathrooms,0.34917,0.0,6.0
full_bathrooms,1.488231,1.0,8.0
age_in_months,52.915376,0.0,255.0
parking_lots,1.135525,0.0,8.0
...,...,...,...
ppsm,14790.199894,4000.0,270597.014925
quarters_since_first_appraisal,9.948678,0.0,20.010965
has_elevator,0.021103,0.0,1.0
is_new,0.469135,0.0,1.0


## One hot encoding (stood here)

todo: subset only the most important features and elimnate the rest (redundant for the recats)

In [ ]:
# categories
columns_categorical = list(set([
    'city_cluster',
    'id_entidad_f',
    'id_tipo_inmueble',
    'ocean_label',
    'tipo_vialidad'
    ]))

# convert to category
df_properties[columns_categorical] = df_properties[columns_categorical].astype('category')


In [ ]:
# one hot encoding
columns_to_one_hot = [
    'id_entidad_f',
    'id_tipo_inmueble'
]

# one hot encoding
df_properties = pd.get_dummies(df_properties, columns=columns_to_one_hot, drop_first=True, dtype='int')

# see columns
df_properties.columns

## Split Data

In [ ]:
# get important columns
cols_for_model = [
    'id_clase_inmueble',
    'edad_anios',
    'cve_vigilancia', 
    'superficie_accesoria',
    'saleable_area',
    'count_supermarkets_at_1km',
    'count_hospitals_at_5km',
    'count_metro_at_1km',
    'count_schools_at_1km',
    'count_restaurants_at_1km',
    'ing_cor',
    'competitors_weighted_mean_log_price_per_sqm',
    'competitors_weighted_mean_log_price_per_sqm_lower',
    'competitors_weighted_mean_log_price_per_sqm_upper',
    'mean_log_valor_fisico_terreno_m2',
    'mean_log_valor_fisico_terreno_m2_lower',
    'mean_log_valor_fisico_terreno_m2_upper', 
    'quarters_since_first_appraisal',
    'has_elevator',
    'is_new',
    'recamaras_recat',
    'banos_recat',
    'medio_banos_recat',
    'estacionamiento_recat',
    'conservacion_recat',
    'cve_vigilancia_recat',
    'regimen_propiedad_colectiva',
    'distance_to_ocean_recat',
    'superficie_terreno_usable',
    'id_entidad_f_02',
    'id_entidad_f_03',
    'id_entidad_f_04',
    'id_entidad_f_05',
    'id_entidad_f_06',
    'id_entidad_f_07',
    'id_entidad_f_08',
    'id_entidad_f_09',
    'id_entidad_f_10',
    'id_entidad_f_11',
    'id_entidad_f_12',
    'id_entidad_f_13',
    'id_entidad_f_14',
    'id_entidad_f_15',
    'id_entidad_f_16',
    'id_entidad_f_17',
    'id_entidad_f_18',
    'id_entidad_f_19',
    'id_entidad_f_20',
    'id_entidad_f_21',
    'id_entidad_f_22',
    'id_entidad_f_23',
    'id_entidad_f_24',
    'id_entidad_f_25',
    'id_entidad_f_26',
    'id_entidad_f_27',
    'id_entidad_f_28',
    'id_entidad_f_29',
    'id_entidad_f_30',
    'id_entidad_f_31',
    'id_entidad_f_32',
    'id_tipo_inmueble_3',
    'id_tipo_inmueble_4'
]

X = df_properties[cols_for_model].copy()
y = df_properties['log_price_per_sqm'].copy()

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    random_state=42,
    stratify=df_properties['id_tipo_inmueble_4']  # same proportion of apartments
    )

In [ ]:
# see
X_train

## Fit Lasso

In [ ]:
# # train lasso
# lm_lasso = LassoCV(
#     cv=5,
#     random_state=42,
#     n_jobs=-1,
#     verbose=True,
#     alphas=np.logspace(-6, 2, 100)
# )

# # fit
# lm_lasso.fit(X_train, y_train)

# # see best alpha
# idx_min_mse = np.argmin(lm_lasso.mse_path_.mean(axis=1))
# min_mse = lm_lasso.mse_path_.mean(axis=1)[idx_min_mse]
# min_mse

In [ ]:
def visualize_beta_decay(X, y, n_alphas=100, n_highlight=5, scale=True):
    """
    Visualize how betas decay as alpha increases using lasso_path.
    Highlights top performing features by their index.
    Uses gray for non-highlighted features and adds a black line at y=0.
    """ 
    if scale:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        alphas, coefs, _ = lasso_path(X_scaled, y, n_alphas=n_alphas)
    else:
        alphas, coefs, _ = lasso_path(X, y, n_alphas=n_alphas)
    
    # Convert coefficients to a DataFrame for easier manipulation
    coef_df = pd.DataFrame(coefs.T, columns=[f"Feature {i}" for i in range(coefs.shape[0])])
    coef_df.index = alphas

    # Plot
    _, ax = plt.subplots(figsize=(8, 6))
    
    alpha_min = coef_df.index.min()
    top_coefs = coef_df.loc[alpha_min].map(abs).sort_values().tail(n_highlight)
    
    # Plot non-highlighted features in gray
    for feature in coef_df.columns:
        if feature not in top_coefs.index:
            ax.semilogx(coef_df.index, coef_df[feature], "-", color='gray', alpha=0.5)
    
    # Plot highlighted features in color
    for feature in top_coefs.index:
        ax.semilogx(coef_df.index, coef_df[feature], "-", label=feature)
        feature_index = int(feature.split()[-1])  # Extract feature index
        plt.text(
            alpha_min, 
            coef_df.loc[alpha_min, feature],
            f"f.{feature_index}", 
            horizontalalignment="right", 
            verticalalignment="center"
        )

    # Add black horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

    # Format plot
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.grid(True)
    ax.set_xlabel("Alpha")
    ax.set_ylabel("Coefficient")
    ax.set_title("Beta Decay with Increasing Alpha (using lasso_path)")
    plt.tight_layout()
    plt.show()

    # print the name of the features (number, name)
    for feature in top_coefs.index:
        feature_index = int(feature.split()[-1])  # Extract feature index
        print(f"f.{feature_index}: {X.columns[feature_index]}")


In [ ]:
# visualize scaled
visualize_beta_decay(X_train, y_train, n_highlight=10)

In [ ]:
# visualize not scaled
visualize_beta_decay(
    X_train,
    y_train,
    n_highlight=10,
    scale=False
    )

In [ ]:
X_train['ing_cor'].describe()

---
# Catboost

## Read

In [ ]:
# important columns
cols_to_stay = [
    'observation_id', 'valuation_date', 
    'id_clase_inmueble', 'id_tipo_inmueble', 'conservacion', 'elevador',
    'cve_ref_proximidad_urbana', 'price', 'edad_anios',
    'cve_vigilancia', 'regimen_propiedad', 'tipo_vialidad', 'id_entidad_f',
    'recamaras', 'banos', 'medio_banos',
    'estacionamiento', 'superficie_terreno', 'superficie_construida',
    'superficie_accesoria', 'saleable_area',
    'city_cluster', 'distance_to_ocean', 'ocean_label', 'longitud', 'latitud',
    'count_supermarkets_at_1km', 'count_hospitals_at_5km',
    'count_metro_at_1km', 'count_schools_at_1km', 'count_restaurants_at_1km',
    'ing_cor', 'competitors_weighted_mean_log_price_per_sqm',
    'competitors_weighted_mean_log_price_per_sqm_lower',
    'competitors_weighted_mean_log_price_per_sqm_upper',
    'mean_log_valor_fisico_terreno_m2',
    'mean_log_valor_fisico_terreno_m2_lower',
    'mean_log_valor_fisico_terreno_m2_upper'
]

# read database
df_properties = pd.read_parquet("../../data/clean/properties_shif.parquet").loc[:, cols_to_stay]

# set observation_id as index
df_properties = df_properties.set_index('observation_id')

# see
print(df_properties.shape)
df_properties.head(2)

## Wrangling

### NaNs

In [ ]:
# nan values
df_properties.isna().mean()[df_properties.isna().mean().gt(0)]

In [ ]:
# fill with 0's
df_properties['elevador'] = df_properties['elevador'].fillna(0)
df_properties['cve_vigilancia'] = df_properties['cve_vigilancia'].fillna(0)
df_properties['tipo_vialidad'] = df_properties['tipo_vialidad'].fillna(0)

In [ ]:
# competitors with terrain value
df_properties['competitors_weighted_mean_log_price_per_sqm'] = df_properties['competitors_weighted_mean_log_price_per_sqm'].combine_first(df_properties['mean_log_valor_fisico_terreno_m2'])
df_properties['competitors_weighted_mean_log_price_per_sqm_lower'] = df_properties['competitors_weighted_mean_log_price_per_sqm_lower'].combine_first(df_properties['mean_log_valor_fisico_terreno_m2_lower'])
df_properties['competitors_weighted_mean_log_price_per_sqm_upper'] = df_properties['competitors_weighted_mean_log_price_per_sqm_upper'].combine_first(df_properties['mean_log_valor_fisico_terreno_m2_upper'])

In [ ]:
# nan values
df_properties.isna().mean()[df_properties.isna().mean().gt(0)]

### Casting

In [ ]:
# integer columns
columns_to_integer = [
    'cve_vigilancia', 'tipo_vialidad', 
]

# to int (downcast)
df_properties[columns_to_integer] = df_properties[columns_to_integer].astype('float').round().astype('Int64')

## Feature Engineering

In [ ]:
# params
first_date_obs = df_properties['valuation_date'].min()
last_date_obs = df_properties['valuation_date'].max()

# create columns
df_properties = (
    df_properties
    .assign(
        # objective variable
        ppsm=lambda x: x['price'] / x['saleable_area'],
        log_price_per_sqm=lambda x: np.log(x['ppsm']),
        # quarters since appraisal
        quarters_since_first_appraisal=lambda x: (x['valuation_date'] - first_date_obs).dt.days / (30.4 * 3),
        # recategorizations
        conservacion_recat=lambda x: x['conservacion'].replace({7: 3.5}) - x['conservacion'].min(),
        cve_vigilancia_recat=lambda x: np.where(x['cve_vigilancia'].eq(2), 1, 0),
        superficie_terreno_usable=lambda x: np.where(
            x['id_tipo_inmueble'].eq(4),
            x['superficie_accesoria'],
            x['superficie_terreno'] + x['superficie_accesoria']
        ),
    )
)

In [ ]:
# see new var
df_properties['quarters_since_first_appraisal'].describe()

## Categorical Variables

In [ ]:
# get categorical columns
categorical_columns = [
    'id_tipo_inmueble', 'cve_vigilancia_recat', 'regimen_propiedad', 'id_entidad_f', 
    'city_cluster', 'ocean_label', 'tipo_vialidad'
]

# to category
df_properties[categorical_columns] = df_properties[categorical_columns].astype('category')

## Split Data

In [ ]:
# get important columns
cols_for_model = list(set([
    'id_clase_inmueble',
    'id_tipo_inmueble',
    'elevador',
    'cve_ref_proximidad_urbana',
    'edad_anios',
    'regimen_propiedad',
    'tipo_vialidad',
    'id_entidad_f',
    'recamaras',
    'banos',
    'medio_banos',
    'estacionamiento',
    'superficie_accesoria',
    'saleable_area',
    'city_cluster',
    'distance_to_ocean',
    'ocean_label',
    'longitud',
    'latitud',
    'count_supermarkets_at_1km',
    'count_hospitals_at_5km',
    'count_metro_at_1km',
    'count_schools_at_1km',
    'count_restaurants_at_1km',
    'ing_cor',
    'competitors_weighted_mean_log_price_per_sqm',
    'competitors_weighted_mean_log_price_per_sqm_lower',
    'competitors_weighted_mean_log_price_per_sqm_upper',
    'mean_log_valor_fisico_terreno_m2',
    'mean_log_valor_fisico_terreno_m2_lower',
    'mean_log_valor_fisico_terreno_m2_upper',
    'quarters_since_first_appraisal',
    'conservacion_recat',
    'cve_vigilancia_recat',
    'superficie_terreno_usable'
]))

X = df_properties[cols_for_model].copy()
y = df_properties['ppsm'].copy()

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    random_state=42,
    stratify=df_properties['id_tipo_inmueble'].eq(4)  # same proportion of apartments
    )

# pool
train_pool = Pool(X_train, y_train, cat_features=categorical_columns)
test_pool = Pool(X_test, y_test, cat_features=categorical_columns)

## Fit Catboost

In [ ]:
# fit catboost
cb_model = CatBoostRegressor(
    loss_function='MAE',
    eval_metric='MAPE',
    verbose=100,
    random_seed=42
)

# fit
cb_model.fit(train_pool)

In [ ]:
# explainer
explainer = shap.TreeExplainer(cb_model)

# shap values
shap_values = explainer(X_train)

In [ ]:
# plot feature importance
shap.plots.bar(shap_values, max_display=20)

In [ ]:
# Plot a beeswarm plot of SHAP values
shap.plots.beeswarm(shap_values, max_display=20)

---
# Sandbox

In [20]:
df_properties.query("property_type_id.eq(4) & land_area.le(10)").describe()

/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ravj/opt/anaconda3/envs/mds-research-stay/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,price,half_bathrooms,full_bathrooms,age_in_months,parking_lots,valuation_date,level,total_levels,bedrooms,built_area,saleable_area,land_area,remaining_useful_life,count_supermarkets_at_1km,count_hospitals_at_5km,count_metro_at_1km,count_schools_at_1km,count_restaurants_at_1km,tam_loc,est_socio,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,mayores,menores,p12_64,p65mas,percep_ing,ing_cor,estim_alqu,otros_ing,gasto_mon,vivienda,alquiler,pred_cons,cuidados,tipo_viv,antiguedad,cuart_dorm,tenencia,renta,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,escrituras,tot_resid,tot_hom,tot_muj,tot_hog,total_viviendas,ppsm_terrain,ppsm,quarters_since_first_appraisal,has_elevator,is_new
count,1.138200e+04,11382.000000,11382.000000,11382.000000,11382.000000,11382,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,1.138200e+04,1.138200e+04,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,1.138200e+04,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000,11382.000000
mean,2.562701e+06,0.040503,1.407046,19.292216,0.897645,2021-06-26 22:28:16.573537024,9.268406,3.022052,1.838166,59.635829,60.552803,7.612546,68.464769,2.214110,13.062643,0.507556,0.314971,14.928659,1.166992,2.750000,2.000351,1.073625,inf,inf,3.154297,2.712891,0.442627,2.361328,0.350586,2.230469,82955.187500,12726.172852,73.065292,49957.480469,7575.182617,4586.655273,713.917664,2425.216064,1.280273,inf,2.072266,3.369179,1580.262573,4338.057617,385.644440,0.364417,1.156475,0.135010,4.164997,1.105605,3.205078,1.513672,1.690430,1.015625,177659.553593,27424.179845,39426.959856,9.955442,0.255315,0.697417
min,2.700000e+05,0.000000,1.000000,0.000000,0.000000,2019-01-02 00:00:00,0.000000,0.000000,1.000000,26.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,1.000000,3.812500e+01,2.949219e+00,2.441406,1.858398,0.180054,1.358398,0.000000,1.307617,22246.158203,2876.163330,0.000000,15795.153320,1295.720581,0.000000,0.000000,606.029846,1.000000,6.351562e+00,1.428711,1.000000,0.000000,978.804138,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,2.464844,1.057617,1.057617,1.000000,1404.000000,1343.000000,4912.500000,0.010965,0.000000,0.000000
25%,6.740000e+05,0.000000,1.000000,0.000000,0.000000,2020-02-06 00:00:00,3.000000,1.000000,2.000000,44.000000,46.000000,6.000000,60.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.482422,2.000000,1.000000,4.915625e+01,6.605469e+00,3.017578,2.554688,0.312744,2.207031,0.232666,2.132812,59297.687500,7233.132324,4.605845,38126.960938,4776.178223,2082.843750,212.812424,1530.793091,1.000000,1.492188e+01,1.885986,4.000000,767.280029,2457.317139,171.139420,0.237992,1.000000,0.088074,5.000000,1.000000,3.025391,1.363281,1.549805,1.000000,96192.000000,4323.000000,14270.833333,4.396930,0.000000,0.000000
50%,2.271500e+06,0.000000,1.000000,0.000000,1.000000,2021-06-24 00:00:00,7.000000,1.000000,2.000000,56.000000,56.000000,8.000000,70.000000,2.000000,14.000000,0.000000,0.000000,4.000000,1.000000,2.845703,2.000000,1.000000,5.375000e+01,6.917969e+00,3.214844,2.705078,0.423584,2.373047,0.363037,2.244141,76742.710938,11910.004883,42.652428,47140.269531,6524.252930,3037.980225,521.791138,1975.968384,1.000000,2.492188e+01,2.169922,4.000000,1100.134521,4065.352539,303.669769,0.336579,1.000000,0.121277,5.000000,1.000000,3.255859,1.548828,1.749023,1.008789,178716.000000,20369.000000,38745.129870,9.923246,0.000000,1.000000
75%,3.659800e+06,0.000000,2.000000,12.000000,1.000000,2022-10-19 00:00:00,13.00